In [ ]:
import requests
import os
import sys
import urllib.request
import json
import csv
import pandas as pd
import numpy as np

In [ ]:
food_type = pd.read_csv('food_type_final_v2.csv', encoding='cp949')
food_name = food_type[['Attraction']].copy()
food_name[:3]

Attraction
0        채그로
1     소소한 풍경
2    마복림할머니집

In [ ]:
tour_type = pd.read_csv('Tour_Type_Final_v2.csv', encoding='cp949')
tour_name = tour_type[['Attraction']].copy()
tour_name[:3]

Attraction
0     104고지전적비
1  10꼬르소꼬모 청담점
2         63아트

In [ ]:
tour_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Attraction  336 non-null    object
dtypes: object(1)
memory usage: 2.8+ KB


In [ ]:
### Naver_API_사진호출

### 사이트 제한 _ Tour

In [ ]:
import os
import urllib.request
import json
import csv
import pandas as pd
import time

def Tour_url_naver(search_name, client_id, client_secret):
    # URL 인코딩을 위해 검색어를 인코딩
    encText = urllib.parse.quote(search_name)
     # 큰 이미지를 필터링하여 검색 URL 생성
    url = f"https://openapi.naver.com/v1/search/image?query={encText}&filter=large"  # 큰 이미지 필터 적용

    # API 요청 생성
    request = urllib.request.Request(url)
    # 헤더에 클라이언트 ID와 시크릿을 추가
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    # API 요청 보내고 응답 받기
    response = urllib.request.urlopen(request)
    # 응답 코드 확인
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        result = json.loads(response_body)

        if 'items' in result:
            # 이미지 리스트 가져오기
            img_list = result['items']
            # 도메인 리스트
            valid_domains = ['seoul.go.kr','google.com', 'daum.net', 'kakao.com', 'naver.com', 'visitkorea.or.kr', 'naver.net', 'pstatic.net','nculture.org','pinterest.com', 'unsplash.com', 'pixabay.com', 'flickr.com', '500px.com']  # 유효 도메인 리스트
            # 그냥하면 네이버 뉴스에서만 수집되어 관계 없는 이미지를 불러옴

            for img in img_list:
                # 뉴스 링크가 아니고, 유효 도메인에 속하는지 확인
                if 'news' not in img['link'] and any(domain in img['link'] for domain in valid_domains):  # 조건
                    first_img_link = img['link']
                    print(f"{search_name}: {first_img_link}")
                    return first_img_link

            # 유효한 이미지를 찾지 못했을 때
            print(f"{search_name}: No valid images found from specified domains")
            return None
        else:
            # 'items' 키가 응답에 없을 때
            print(f"{search_name}: Key 'items' not found in the response")
            return None
    else:
        # 에러 코드 출력
        print(f"{search_name}: Error Code: {rescode}")
        return None

# 발급받은 아이디 비번
client_id = "클라이언트 아이디"
client_secret = "클라이언트 비밀번호"
savePath = "저장위치"


# CSV 파일에 링크 저장
with open(savePath, mode='w', newline='', encoding='cp949') as file:
    writer = csv.writer(file)
    #첫 행 컬럼 이름 작성
    writer.writerow(['Attraction', 'Image Link'])

    # 관광지 이름 리스트에서 각각의 관광지 이름에 대해 반복
    for attraction in tour_name['Attraction']:
        # 네이버 API를 사용하여 이미지 링크를 검색
        image_link = Tour_url_naver(attraction, client_id, client_secret)
        if image_link:
            writer.writerow([attraction, image_link])  #이미지 링크가 존재 하면 관광지명과 링크 저장
        else:
            writer.writerow([attraction, ""])  # 이미지 링크가 없을 경우 빈 값 저장
        time.sleep(2)  # 페이지에 접속하지 못할 수도 잇음으로..혹시나..2초 대기

# 저장 완료 시 출력
print("--------CSV file created successfully--------")

104고지전적비: https://dbscthumb-phinf.pstatic.net/2765_000_13/20230216075221780_Y9J8XG04I.jpg/12856082.jpg?type=m4500_4500_fst
10꼬르소꼬모 청담점: https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=30085325-77d2-44b5-9a0d-fb72f8f5456d
63아트: http://post.phinf.naver.net/MjAxODA5MTdfMTQx/MDAxNTM3MTQ4NDE5NTQ0.TSCv5nm4zuwmhyh4sL--89n9Tfm2JMtnB9PKUtOcflEg.jaHQ6PXMuKuob2h_juBHFy0FKCfVoAN5E80GAnaKx7Ig.JPEG/ICvdB9uiQ_oNGy7MC44bhtPHzzIA.jpg
가락농수산물종합도매시장: https://dbscthumb-phinf.pstatic.net/2644_000_4/20190801101957584_255I4R616.jpg/ed673e3e-66d3-4e.jpg?type=m935_fst_nce
가락몰: http://post.phinf.naver.net/MjAxNzA1MTRfMjM4/MDAxNDk0NzUwMjUxMjQw.9MXJQW191VuZa_ASFG3AOLSnngeV-CjPD_kJoGmRhy8g.ytoSGjoYKyj9xjKNzN0u69hXgXtXidlBZbdPyjt75G8g.JPEG/Ih47Ypd1A7xcC2awCTmHDzo1jogg.jpg
가회민화박물관: https://dbscthumb-phinf.pstatic.net/2644_000_6/20200217211640183_WEAHN7FG6.jpg/281f513e-a816-4c.jpg?type=m935_fst_nce
간송미술관: https://minio.nculture.org/amsweb-opt/multimedia_assets/106/85028/84252/c/%EC%84%9C%EC%9A%B8-%EB%B3%B4%ED%99%94%

### 사이트제한_FOOD

In [ ]:
import os
import urllib.request
import json
import csv
import pandas as pd
import time

def food_url_naver(search_name, client_id, client_secret):
    encText = urllib.parse.quote(search_name)
    url = f"https://openapi.naver.com/v1/search/image?query={encText}&filter=large"  # 큰 이미지 필터 적용

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        result = json.loads(response_body)

        if 'items' in result:
            img_list = result['items']
            valid_domains = ['seoul.go.kr','google.com', 'daum.net', 'kakao.com', 'naver.com', 'visitkorea.or.kr', 'naver.net', 'pstatic.net','pinterest.com', 'unsplash.com', 'pixabay.com', 'flickr.com', '500px.com']  # 유효 도메인 리스트

            for img in img_list:
                if 'news' not in img['link'] and any(domain in img['link'] for domain in valid_domains):
                    first_img_link = img['link']
                    print(f"{search_name}: {first_img_link}")
                    return first_img_link

            print(f"{search_name}: No valid images found from specified domains")
            return None
        else:
            print(f"{search_name}: Key 'items' not found in the response")
            return None
    else:
        print(f"{search_name}: Error Code: {rescode}")
        return None

client_id = "클라이언트 아이디"
client_secret = "클라이언트 비밀번호"
savePath = "저장위치"


# CSV 파일에 링크 저장
with open(savePath, mode='w', newline='', encoding='cp949') as file:
    writer = csv.writer(file)
    writer.writerow(['Attraction', 'Image Link'])

    for attraction in food_name['Attraction']:
        image_link = food_url_naver(attraction, client_id, client_secret)
        if image_link:
            writer.writerow([attraction, image_link])
        else:
            writer.writerow([attraction,""]) # 이미지 링크가 없을 경우 빈 값 저장
        time.sleep(2)  # 1초 대기

# 저장 완료 시 출력
print("--------CSV file created successfully--------")


채그로: https://modo-phinf.pstatic.net/20220808_69/1659931089730OV9oT_JPEG/mosaVi32U7.jpeg
소소한 풍경: https://tong.visitkorea.or.kr/cms/resource/50/1922450_image2_1.jpg
마복림할머니집: https://ldb-phinf.pstatic.net/20231225_212/1703468172520VQMek_JPEG/KakaoTalk_20231225_103343760.jpg
뮤직컴플렉스 서울: https://ldb-phinf.pstatic.net/20220629_36/1656480012677mq1Gl_JPEG/KakaoTalk_20220628_015839552_19.jpg
또순이네집: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fpup-review-phinf.pstatic.net%2FMjAyMzAzMjZfOTkg%2FMDAxNjc5Nzk2MDcwNDIz.fNRbkusPo52RQuZOjVIMhAf4touuDcma2EtHpsMaCnog.UOTiJdc3Ancx5lsXlQ0rSXNJSCWVcC-syC5lmfThUZcg.JPEG%2F96DE82CF-5CF8-4CCF-85F1-04A1B7774BFC.jpeg
우묵배미: No valid images found from specified domains
토속촌삼계탕: http://cfile220.uf.daum.net/image/2744804A53CD344E1AF2FE
석파랑: https://mediahub.seoul.go.kr/uploads/mediahub/2021/02/aa7f3d1bf4a6480a9a0b3598a7a05baa.jpg
선천집: http://post.phinf.naver.net/MjAxODExMDJfMTQw/MDAxNTQxMTM5NjExOTY1.IlGTUcWt103SZdIK_bJIQjnlQErE